In [66]:
import pandas as pd
from bs4 import BeautifulSoup
from matplotlib import pyplot as plt
import seaborn as sns
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense,Embedding,Flatten, LSTM,BatchNormalization
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
# !pip install emoji
import emoji
import re
from nltk.corpus import stopwords
import nltk
# nltk.download('stopwords')
import spacy
nlp = spacy.load('en_core_web_sm')
from sklearn.model_selection import train_test_split
import numpy as np
StopWords = stopwords.words('english')
from sklearn.utils import class_weight
import os
from keras.models import load_model
import datetime

In [67]:
try:
 device_name = os.environ['COLAB_TPU_ADDR']
 TPU_ADDRESS = 'grpc://' + device_name
 print('Found TPU at: {}'.format(TPU_ADDRESS))
except KeyError:
 print('TPU not found')

Found TPU at: grpc://10.8.250.90:8470


In [68]:
df = pd.read_csv('IMDB Dataset.csv',error_bad_lines=False, engine="python")

<ipython-input-68-80cfbe6f977a>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('IMDB Dataset.csv',error_bad_lines=False, engine="python")
Skipping line 34833: unexpected end of data


In [69]:
X_train, X_test, y_train, y_test = train_test_split(df['review'], df['sentiment'],test_size=0.25)

##Preprocessing


In [71]:

def text_preprocess(text_):
  out = []
  text_ = text_.lower()  ## lower case

  pattern = re.compile(r'https?://\S+|www\.\S+') ## remove URLs
  text_ = pattern.sub(r'', text_)

  text_ = BeautifulSoup(text_).get_text() ## remove html tags

  text_ = emoji.demojize(text_)  ##convert emojis to text

  new_text = []                  ## remove stop words
  for word in text_.split():
      if word in StopWords:
          new_text.append('')
      else:
          new_text.append(word)
  text_= " ".join(new_text)

  doc = nlp(text_)         ## Lemmatization
  text_= " ".join([token.lemma_ for token in doc])

  return re.sub(' +', ' ', text_)

X_train =  X_train.apply(text_preprocess).to_numpy()
y_train = y_train.apply(text_preprocess).to_numpy()

<ipython-input-71-26cefacf822f>:8: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text_ = BeautifulSoup(text_).get_text() ## remove html tags


In [73]:
vocab_size=15000
output_dim = 20
max_length = 700
trunc_type='post'
padding_type='post'
learning_rate = 0.001

## tokenization
tokenizer = Tokenizer(num_words = vocab_size, oov_token = "<OOV>", lower = False)
tokenizer.fit_on_texts(X_train)

X_train_clean = np.array(tokenizer.texts_to_sequences(X_train))
X_test_clean = np.array(tokenizer.texts_to_sequences(X_test))


## padding
X_train_clean = pad_sequences(X_train_clean, maxlen=max_length, padding=padding_type, truncating=trunc_type)
X_test_clean = pad_sequences(X_test_clean, maxlen=max_length, padding=padding_type, truncating=trunc_type)


## Tokenize and remove special characters
map_dict = {'positive':1,
            'negative':0}

y_train_coded = np.array(list(map(lambda x: map_dict[x],y_train)))
y_test_coded = np.array(list(map(lambda x: map_dict[x],y_test)))

## Class Wieght
# class_weights = class_weight.compute_class_weight(
#                                         class_weight = "balanced",
#                                         classes = np.unique(y_train_coded),
#                                         y = y_train_coded
#                                     )
# class_weights = dict(zip(np.unique(y_train_coded), class_weights))

tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
tf.tpu.experimental.initialize_tpu_system(tpu)
tf.config.run_functions_eagerly(False)
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)
strategy = tf.distribute.TPUStrategy(tpu)

with strategy.scope():
    model = Sequential()
    model.add(Embedding(input_dim = vocab_size,output_dim=output_dim,input_length=max_length))
    model.add(BatchNormalization())
    model.add(LSTM(128,return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(BatchNormalization())
    model.add(LSTM(128,return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model.summary()

<ipython-input-73-7a075ffef155>:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_train_clean = np.array(tokenizer.texts_to_sequences(X_train))
<ipython-input-73-7a075ffef155>:13: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X_test_clean = np.array(tokenizer.texts_to_sequences(X_test))


Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_12 (Embedding)    (None, 700, 20)           300000    
                                                                 
 batch_normalization_36 (Bat  (None, 700, 20)          80        
 chNormalization)                                                
                                                                 
 lstm_24 (LSTM)              (None, 700, 128)          76288     
                                                                 
 batch_normalization_37 (Bat  (None, 700, 128)         512       
 chNormalization)                                                
                                                                 
 lstm_25 (LSTM)              (None, 128)               131584    
                                                                 
 batch_normalization_38 (Bat  (None, 128)            

In [75]:
history = model.fit(X_train_clean, y_train_coded,epochs=50,validation_data=(X_test_clean,y_test_coded),  verbose = 2) ##class_weight = class_weights,
name = 'IMDB_sentiment_Analysis_{}.h5'.format(datetime.datetime.now().strftime('%dth_%B_%H_%M_%S'))
model.save(name)  # creates a HDF5 file 'my_model.h5'
# del model  # deletes the existing model

Epoch 1/50
817/817 - 182s - loss: 0.7200 - acc: 0.5009 - val_loss: 0.8774 - val_acc: 0.4990 - 182s/epoch - 222ms/step
Epoch 2/50
817/817 - 151s - loss: 0.6967 - acc: 0.4983 - val_loss: 0.7070 - val_acc: 0.5006 - 151s/epoch - 185ms/step
Epoch 3/50
817/817 - 151s - loss: 0.6973 - acc: 0.5019 - val_loss: 0.7846 - val_acc: 0.4997 - 151s/epoch - 185ms/step
Epoch 4/50
817/817 - 151s - loss: 0.6944 - acc: 0.4998 - val_loss: 0.7002 - val_acc: 0.4974 - 151s/epoch - 185ms/step
Epoch 5/50
817/817 - 151s - loss: 0.6943 - acc: 0.4962 - val_loss: 0.7273 - val_acc: 0.4994 - 151s/epoch - 184ms/step
Epoch 6/50
817/817 - 151s - loss: 0.6941 - acc: 0.4972 - val_loss: 0.7623 - val_acc: 0.5001 - 151s/epoch - 185ms/step
Epoch 7/50
817/817 - 151s - loss: 0.6940 - acc: 0.4939 - val_loss: 0.7020 - val_acc: 0.5034 - 151s/epoch - 185ms/step
Epoch 8/50
817/817 - 151s - loss: 0.6937 - acc: 0.4986 - val_loss: 1.0242 - val_acc: 0.4954 - 151s/epoch - 184ms/step
Epoch 9/50
817/817 - 151s - loss: 0.6939 - acc: 0.4974 -

In [115]:
# model.predict('Movie was really great!')
def model_test(input_text):
  if type(input_text)==
  input_text = text_preprocess(input_text)
  input_text_clean = np.array(tokenizer.texts_to_sequences([input_text]))

  ## padding
  input_text_padded = pad_sequences(input_text_clean, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  return input_text_padded

model.predict(model_test('Movie was really good but ac was too cold and irritating!'))
# returns a compiled model
# identical to the previous one
# model = load_model('my_model.h5')


1/1 [==============================] - 1s 692ms/step


array([[0.0076232]], dtype=float32)